# **Descripción general del proyecto**

Este proyecto tiene como objetivo desarrollar un **pipeline ETL completo** para el análisis del consumo de energía eléctrica en una cadena de tiendas físicas en distintos países de Latinoamérica, utilizando datos operativos reales simulados a nivel diario.

A lo largo del proyecto se trabaja con un dataset de alta granularidad que contiene información sobre consumo energético, demanda eléctrica, características físicas de las tiendas y su estado operativo. El enfoque principal es **limpiar, transformar y enriquecer los datos**, garantizando su calidad antes de realizar cualquier análisis agregado.

El proyecto se implementa utilizando Pandas, PySpark y SQL, permitiendo comparar enfoques y reforzar el entendimiento de los principios fundamentales de ETL, tales como:

* orden correcto del flujo de transformación,
* manejo de valores nulos con criterio de negocio,
* creación de métricas derivadas relevantes,
* uso adecuado de filtros antes y después de agregaciones.

Este trabajo está diseñado tanto como ejercicio de aprendizaje profundo como **proyecto demostrable de portafolio** orientado a análisis de datos y data engineering.

## Contexto del proyecto

Este notebook implementa un pipeline de **ETL distribuido utilizando PySpark**, aplicado a datos diarios de consumo de energía eléctrica en una cadena de tiendas.

El objetivo principal es replicar y escalar las transformaciones realizadas en Pandas, utilizando operaciones propias de Spark como:

- filtros distribuidos,
- imputaciones basadas en agregaciones por grupo,
- creación de columnas derivadas,
- validación de calidad de datos.

El proyecto enfatiza el uso correcto del **orden de las transformaciones**, la separación entre limpieza y análisis, y el manejo de datos faltantes con criterios realistas de negocio.
Este enfoque es representativo de escenarios reales en plataformas de big data como Databricks.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, coalesce


# INICIALIZACIÓN DE SPARK
spark = (
    SparkSession
        .builder
        .appName('elt_energia_electrica')
        .getOrCreate()
)


# CARGA DEL DATASET
df = spark.read.csv(
    '/Volumes/workspace/default/raw_data/consumo_energia_tiendas_diario_2023_2024.csv',
    header=True,
    inferSchema=True
)


# ESQUEMA DE LOS DATOS
df.show()


# INFORMACIÓN DEL DATASET
df.printSchema()


# VERIFICAMOS NULOS EN COLUMNAS
for columna in df.columns:
    print(
        f"Nulos en {columna}: ",
        df.filter(col(columna).isNull()).count()
    )


'''
Aunque las tiendas presentan consumo eléctrico en días cerrados debido a cargas base y sistemas auxiliares, estos registros no representan operación comercial normal. Para evitar distorsionar métricas de eficiencia energética, se excluyen de los análisis operativos.

Existen 5482 registros nulos en las columnas de area_m2, consumo_kwh y demanda_kw. Ya que es muy importante conservar la información completa 
de estas columnas, imputaremos los valores utilizando el promedio de los datos, considerando el tipo de tienda y el país donde se encuentran 
localizadas. 
'''


# FILTRADO DE TIENDAS ACTIVAS
df_active = df.filter(col("estado_tienda") == 'Activa')


# DETERMINAMOS PROMEDIO DE DATOS
promedio = (
    df_active
        .groupBy(['pais', 'tipo_tienda'])
        .agg(
            avg(col('area_m2')).alias('promedio_area_m2'),
            avg(col('consumo_kwh')).alias('promedio_consumo_kwh'),
            avg(col('demanda_kw')).alias('promedio_demanda_kw')
        )
)

promedio.show()


# UNIMOS PROMEDIOS AL DATASET ACTIVO
df_active = df_active.join(
    promedio,
    on=['pais', 'tipo_tienda'],
    how='left'
)


# IMPUTAMOS VALORES EN COLUMNAS OBJETIVO
df_active = df_active.withColumn(
    "area_m2",
    coalesce(col('area_m2'), col("promedio_area_m2"))
)

df_active = df_active.withColumn(
    "consumo_kwh",
    coalesce(col('consumo_kwh'), col("promedio_consumo_kwh"))
)

df_active = df_active.withColumn(
    "demanda_kw",
    coalesce(col('demanda_kw'), col("promedio_demanda_kw"))
)


# ELIMINAMOS COLUMNAS AUXILIARES
df_active = df_active.drop(
    "promedio_area_m2",
    "promedio_consumo_kwh",
    "promedio_demanda_kw"
)


# CREACIÓN DE KPIs
df_active = df_active.withColumn(
    'kpi_kwh/m2',
    col("consumo_kwh") / col("area_m2")
)

df_active = df_active.withColumn(
    'kpi_kw/m2',
    col("demanda_kw") / col("area_m2")
)


# VERIFICAMOS NULOS EN COLUMNAS
for columna in df_active.columns:
    print(
        f"Nulos en {columna}: ",
        df_active.filter(col(columna).isNull()).count()
    )


In [0]:
df_active.show()